### 비율 검정
- z-test : proportions_ztest는 두 집단의 '총 횟수'와 '성공 횟수'라는 요약본(배열)을 받아 비율의 차이를 판정한다.
- 양, 단측 검정

In [42]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# A: 10번 중 8번 성공(0.8), B: 20번 중 12번 성공(0.6)
count = np.array([64,284]) # 성공률
nobs = np.array([89,802]) # 총횟수

stat_two, p_two = proportions_ztest(count=count, nobs=nobs, alternative="two-sided") # 🐚 통계수치,p-value = proportions_ztest(성공률,총횟수,alternative=)
stat_l, p_l = proportions_ztest(count=count, nobs=nobs, alternative="larger") # 🐚 alternative는 상위볼떄 Large 하위는 smaller 양쪽은 two-sided
stat_s, p_s = proportions_ztest(count=count, nobs=nobs, alternative="smaller")

print("two-sided p:", p_two)
print("larger    p:", p_l)
print("smaller   p:", p_s)


two-sided p: 2.143654889813421e-11
larger    p: 1.0718274449067105e-11
smaller   p: 0.9999999999892817


# 클로드

## 1. 가설세우기
### 비즈니스 질문: "VIP 고객이 일반 고객보다 생존률이 높은가?"
- H0 (귀무가설): VIP와 일반의 생존률은 같다 (차이 = 0)
- H1 (대립가설): VIP와 일반의 생존률은 다르다 (차이 ≠ 0)

### 왜 H0가 "같다"인가?
- → 검사가 "무죄 추정"부터 시작하듯, 통계도 "효과 없음"부터 시작
- → 충분한 증거가 있어야만 "효과 있다"고 인정

## Step 2: 데이터 준비

In [ ]:
import pandas as pd
import numpy as np

# 타이타닉 데이터 로드
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# Fare 상위 10%를 VIP로 정의
threshold = df['Fare'].quantile(0.90)
df['is_vip'] = (df['Fare'] >= threshold).astype(int)

# 두 그룹 분리
vip = df[df['is_vip'] == 1]['Survived']      # VIP 그룹의 생존 여부
normal = df[df['is_vip'] == 0]['Survived']   # 일반 그룹의 생존 여부

print(f"VIP 그룹: {len(vip)}명, 생존률: {vip.mean():.1%}")
print(f"일반 그룹: {len(normal)}명, 생존률: {normal.mean():.1%}")

# 여기서 눈에 보이는 차이: 72.3% - 35.4% = 36.9%p
# 질문: 이 36.9%p 차이가 진짜야, 운이야?

VIP 그룹: 90명, 생존률: 76.7%
일반 그룹: 801명, 생존률: 34.1%


## Step 3: 검정 실행 (비율 검정)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# 입력 데이터 만들기
# count = [VIP 생존자 수, 일반 생존자 수]
# nobs = [VIP 총 인원, 일반 총 인원]

count = np.array([vip.sum(), normal.sum()])  # [64, 284] 같은 형태
nobs = np.array([len(vip), len(normal)])     # [89, 802] 같은 형태

print(f"VIP: {count[0]}명 생존 / {nobs[0]}명 = {count[0]/nobs[0]:.1%}")
print(f"일반: {count[1]}명 생존 / {nobs[1]}명 = {count[1]/nobs[1]:.1%}")

# Z-test 실행
z_stat, p_value = proportions_ztest(count, nobs)

print(f"\n검정통계량(z): {z_stat:.3f}")
print(f"p-value: {p_value:.6f}")

# **예상 출력:**
# ```
# VIP: 64명 생존 / 89명 = 71.9%
# 일반: 284명 생존 / 802명 = 35.4%

# 검정통계량(z): 6.847
# p-value: 0.000000  (실제로는 7.5e-12 같은 극소값)

VIP: 69명 생존 / 90명 = 76.7%
일반: 273명 생존 / 801명 = 34.1%

검정통계량(z): 7.876
p-value: 0.000000


## Step 4: 판결 내리기


In [ ]:
alpha = 0.05  # 유의수준 (기준선)

if p_value < alpha:
    print(f"✅ p-value({p_value:.4f}) < {alpha}")
    print("→ 귀무가설(H0) 기각!")
    print("→ VIP와 일반의 생존률 차이는 통계적으로 유의미하다.")
else:
    print(f"❌ p-value({p_value:.4f}) >= {alpha}")
    print("→ 귀무가설(H0) 기각 실패")
    print("→ 차이가 우연일 가능성을 배제할 수 없다.")

✅ p-value(0.0000) < 0.05
→ 귀무가설(H0) 기각!
→ VIP와 일반의 생존률 차이는 통계적으로 유의미하다.


## Step 5: 결론 작성법 (실무 템플릿)

In [35]:
# 효과 크기 (얼마나 차이나는지)
effect = vip.mean() - normal.mean()

print("=" * 50)
print("[분석 결론]")
print(f"• 효과 크기: VIP가 일반보다 {effect:.1%}p 높은 생존률")
print(f"• 통계적 유의성: p-value = {p_value:.2e} < 0.05")
print(f"• 샘플 수: VIP {len(vip)}명, 일반 {len(normal)}명")
print(f"• 판정: 차이는 우연이 아니라 유의미함")
print("=" * 50)

[분석 결론]
• 효과 크기: VIP가 일반보다 42.6%p 높은 생존률
• 통계적 유의성: p-value = 3.37e-15 < 0.05
• 샘플 수: VIP 90명, 일반 801명
• 판정: 차이는 우연이 아니라 유의미함


In [43]:
"""
가설검정 실무 템플릿 - 비율 검정 (Z-test)
"""
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.stats.proportion as prop

# ===== 1. 데이터 로드 =====
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# ===== 2. 그룹 정의 =====
threshold = df['Fare'].quantile(0.90)
vip = df[df['Fare'] >= threshold]['Survived']
normal = df[df['Fare'] < threshold]['Survived']

# ===== 3. 기술통계 (효과 크기) =====
print("=" * 60)
print("[Step 1] 기술통계")
print("=" * 60)
effect_size = vip.mean() - normal.mean()
print(f"VIP 생존률: {vip.mean():.1%} (n={len(vip)})")
print(f"일반 생존률: {normal.mean():.1%} (n={len(normal)})")
print(f"차이 (효과 크기): {effect_size:.1%}p")

# ===== 4. 가설 설정 =====
print("\n" + "=" * 60)
print("[Step 2] 가설")
print("=" * 60)
print("H0: VIP 생존률 = 일반 생존률 (차이 없음)")
print("H1: VIP 생존률 ≠ 일반 생존률 (차이 있음)")
print("유의수준 α = 0.05")

# ===== 5. 검정 실행 =====
print("\n" + "=" * 60)
print("[Step 3] 검정 실행")
print("=" * 60)
count = np.array([vip.sum(), normal.sum()])
nobs = np.array([len(vip), len(normal)])

z_stat, p_value = proportions_ztest(count, nobs)
print(f"검정통계량 z = {z_stat:.3f}")
print(f"p-value = {p_value:.2e}")

# ===== 6. 신뢰구간 =====
ci_vip = prop.proportion_confint(vip.sum(), len(vip))
ci_normal = prop.proportion_confint(normal.sum(), len(normal))
print(f"\nVIP 95% CI: [{ci_vip[0]:.1%}, {ci_vip[1]:.1%}]")
print(f"일반 95% CI: [{ci_normal[0]:.1%}, {ci_normal[1]:.1%}]")

# ===== 7. 판정 =====
print("\n" + "=" * 60)
print("[Step 4] 판정")
print("=" * 60)
alpha = 0.05
if p_value < alpha:
    print(f"✅ p-value ({p_value:.2e}) < α ({alpha})")
    print("→ H0 기각: 두 그룹의 생존률 차이는 통계적으로 유의미함")
else:
    print(f"❌ p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("→ H0 기각 실패: 유의미한 차이라고 볼 증거 부족")

# ===== 8. 결론 보고 (한 문장) =====
print("\n" + "=" * 60)
print("[Step 5] 결론")
print("=" * 60)
print(f"""
VIP 그룹(Fare 상위 10%)의 생존률은 {vip.mean():.1%}로, 
일반 그룹 {normal.mean():.1%} 대비 {effect_size:.1%}p 높았으며,
이 차이는 통계적으로 유의미함 (z={z_stat:.2f}, p<0.001).
""")


[Step 1] 기술통계
VIP 생존률: 76.7% (n=90)
일반 생존률: 34.1% (n=801)
차이 (효과 크기): 42.6%p

[Step 2] 가설
H0: VIP 생존률 = 일반 생존률 (차이 없음)
H1: VIP 생존률 ≠ 일반 생존률 (차이 있음)
유의수준 α = 0.05

[Step 3] 검정 실행
검정통계량 z = 7.876
p-value = 3.37e-15

VIP 95% CI: [67.9%, 85.4%]
일반 95% CI: [30.8%, 37.4%]

[Step 4] 판정
✅ p-value (3.37e-15) < α (0.05)
→ H0 기각: 두 그룹의 생존률 차이는 통계적으로 유의미함

[Step 5] 결론

VIP 그룹(Fare 상위 10%)의 생존률은 76.7%로, 
일반 그룹 34.1% 대비 42.6%p 높았으며,
이 차이는 통계적으로 유의미함 (z=7.88, p<0.001).

